Import custom REN module

In [1]:
import torch
from models.REN import REN, t

In order to be able to exclude errors due to numerical precision we use double (Float64) tensors

In [2]:
torch.set_default_dtype(torch.double)

Keep a small network

In [3]:
n_x = 1
n_units = 1
n_u = 1
n_y = 1

Define a simple multiplier

In [4]:
Q = -torch.eye(n_y)
S = torch.zeros(n_y,n_u)
R = torch.eye(n_u)
PI = torch.cat((
        torch.cat((Q,S),dim=1),
        torch.cat((t(S),R),dim=1)
    ), dim=0)
print("The Multiplier:")
print(PI.detach().numpy())

The Multiplier:
[[-1.  0.]
 [ 0.  1.]]


Make one random SISO test, no intial state $x_0 = 0$, no bias $b = 0$

In [5]:
ren = REN(n_x=n_x, n_y=n_y, n_u=n_u, n_units=n_units, Q=Q, S=S, R=R)

In [6]:
u = torch.randn(1,1,1)

In [7]:
y = ren(u)

Verify an IQC given a batch of inputs and outputs

In [8]:
v = torch.cat((y,u), dim=1)
print(v.detach().numpy())

[[[0.09150375]
  [0.45372342]]]


In [9]:
prod = torch.matmul(torch.matmul(t(v), PI), v)
print(prod.detach().numpy())

[[[0.197492]]]


In [10]:
def checkIQC(y,u,Q,S,R):
    PI = torch.cat((
        torch.cat((Q,S),dim=1),
        torch.cat((t(S),R),dim=1)
    ), dim=0)
    v = torch.cat((y,u), dim=1)
    prod = torch.matmul(torch.matmul(t(v), PI), v)
    return torch.all(prod >= 0).item()

Use the built-in function to check the LMI $H(\theta) \succ 0$

In [11]:
print("Matrix H")
print(ren.H.detach().numpy())
print("\nEigenvalues:")
print(torch.linalg.eigvals(ren.H).detach().numpy())

Matrix H
[[ 4.35491143 -2.98325144 -2.34371653]
 [-2.98325144  3.87178169  1.62996617]
 [-2.34371653  1.62996617  3.48239643]]

Eigenvalues:
[8.65767909+0.j 0.99126912+0.j 2.06014136+0.j]


In [12]:
ren.checkLMI()

True

**SOLVED!**: noticed that for small inputs, e.g. $u=10^{-2}$, the IQC is not verified. Need more understanding

Try to randomly pick RENs and due the test

In [13]:
with torch.no_grad():
    for iTest in range(10):
        ren = REN(n_x=n_x, n_y=n_y, n_u=n_u, n_units=n_units, Q=Q, S=S, R=R)
        u2 = torch.tensor([[[1e-2]]])
        y2 = ren(u2)
        if not checkIQC(y2,u2,Q,S,R):
            print("IQC not verified")

Work out one REN

In [14]:
ren = REN(n_x=n_x, n_y=n_y, n_u=n_u, n_units=n_units, Q=Q, S=S, R=R)
u2 = torch.tensor([[[1e-2]]])
y2 = ren(u2)
checkIQC(y2,u2,Q,S,R)

True

For this simple example we can work out the output "manually" as $y = D_{21}\,\sigma(\Lambda^{-1}\, D_{12}\,u)$

In [15]:
torch.matmul(ren.D_21, torch.sigmoid(1/ren.Lambda[0,0]*torch.matmul(ren.D_12, u2)))

tensor([[[-0.5336]]], grad_fn=<TransposeBackward0>)

$D_{21}$ and $D_{12}$ are free matrices. Whereas $\Lambda$ is not.

In [16]:
H_22 = ren.H[ren.n_x:ren.n_x+ren.n_units, ren.n_x:ren.n_x + ren.n_units]

In [17]:
Lambda = 0.5*torch.diag(torch.diag(H_22))
Lambda

tensor([[1.5096]], grad_fn=<MulBackward0>)

In [18]:
ren.Lambda

tensor([[1.5096]], grad_fn=<MulBackward0>)

In [19]:
y2

tensor([[[-0.0013]]], grad_fn=<AddBackward0>)

In [20]:
ren.b

tensor([[0.],
        [0.],
        [0.]])

In [21]:
ren.H

tensor([[11.2621,  3.0948, -0.3593],
        [ 3.0948,  3.0192, -1.2736],
        [-0.3593, -1.2736,  2.2147]], grad_fn=<SubBackward0>)

In [22]:
# Repeat test for many pairs
with torch.no_grad():
    for iTest in range(1000):
        u = torch.randn(1,1,1)
        y = ren(u)
        lmiCheck = ren.checkLMI()
        IQCCheck = checkIQC(y,u,Q,S,R)
        if not lmiCheck:
            print("LMI not satisfied")
        if not IQCCheck:
            print("IQC not satisfied")